In [16]:
import os
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!ls "/content/drive/MyDrive/Colab Notebooks/data_analysis/data"

nyc_taxi_2019-01.csv  nyc_taxi_2020-01.csv


In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/data/nyc_taxi_2019-01.csv')

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/data_analysis/data/nyc_taxi_2019-01.csv'

df = pd.read_csv(filename,
                usecols=['passenger_count',
                         'trip_distance', 'total_amount'],
                dtype={'total_amount':np.float128})

df.head()

,passenger_count,trip_distance,total_amount
0,1,1.5,9.95
1,1,2.6,16.30
2,3,0.0,5.80
3,5,0.0,7.55
4,5,0.0,55.55


In [ ]:
df.sort_values('trip_distance', ascending=False)['total_amount'].iloc[:20].mean()

np.longdouble('290.01000000000000076')

In [ ]:
(
    df
    .sort_values('trip_distance',
                  ascending=False)
    ['total_amount']
    .iloc[:20]
    .mean()
    .round(2)
)

np.longdouble('290.01')

In [ ]:
# first, remove 0-length trips.
df = df[df['trip_distance'] != 0]

# create a new column, in which we calculate the cost per mile
df['cost_per_mile'] = df['total_amount'] / df['trip_distance']

# sort the data frame by that column, get the 5 highest values
df.sort_values('cost_per_mile').tail(5)

<ipython-input-7-c9e7492fd9c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cost_per_mile'] = df['total_amount'] / df['trip_distance']


,passenger_count,trip_distance,total_amount,cost_per_mile
4136499,1,0.01,273.96,27396.000000
6403254,1,0.01,322.30,32230.000000
7099014,4,0.01,415.30,41530.000000
478791,1,0.10,6667.45,66674.500000
2499600,1,2.40,623261.66,259692.358333


In [ ]:
# Remove trips with <2 passengers
df = df[df['passenger_count'] >=2]

# Create a new column based on these values
df['payment_per_person'] = df['total_amount'] / df['passenger_count']

# Find the highest per-person payment
df.sort_values('payment_per_person').tail(10)

<ipython-input-8-79a51cbc907e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['payment_per_person'] = df['total_amount'] / df['passenger_count']


,passenger_count,trip_distance,total_amount,cost_per_mile,payment_per_person
5031491,2,64.30,343.32,5.339347,171.66
4563340,2,0.40,350.30,875.750000,175.15
4202883,2,60.23,369.06,6.127511,184.53
4751745,2,100.78,403.50,4.003771,201.75
5726185,2,65.05,416.82,6.407686,208.41
149362,2,17.20,426.80,24.813953,213.40
7593395,2,83.61,449.32,5.373998,224.66
3842620,2,110.04,515.82,4.687568,257.91
3014027,2,16.60,560.76,33.780723,280.38
2972145,2,19.90,589.96,29.646231,294.98


In [ ]:

filename = '/content/drive/MyDrive/Colab Notebooks/data_analysis/data/nyc_taxi_2019-01.csv'

df = pd.read_csv(filename,
                usecols=['passenger_count',
                         'trip_distance', 'total_amount'])

df.head()

,passenger_count,trip_distance,total_amount
0,1,1.5,9.95
1,1,2.6,16.30
2,3,0.0,5.80
3,5,0.0,7.55
4,5,0.0,55.55


In [ ]:
# For each number of passengers, find the mean cost of a taxi ride.
# Sort this result from lowest (i.e., cheapest) to highest (i.e., most expensive).

df.groupby('passenger_count')['total_amount'].mean()

,total_amount
passenger_count,
0,18.663658
1,15.609601
2,15.831294
3,15.604015
4,15.650307
5,15.546940
6,15.437892
7,48.278421
8,64.105517


In [ ]:
df.groupby('passenger_count')['total_amount'].mean().sort_index()


,total_amount
passenger_count,
0,18.663658
1,15.609601
2,15.831294
3,15.604015
4,15.650307
5,15.546940
6,15.437892
7,48.278421
8,64.105517


In [ ]:
# Now create a new column, `trip_length`, in which the values will be `short`
# (< 2 miles), `medium` (> 2 miles and <= 10 miles), or `long` (> 10 miles).

df['trip_distance_group'] = pd.cut(df['trip_distance'],
                                   [df['trip_distance'].min(), 2, 10,
                                    df['trip_distance'].max()],
                                  labels=['short', 'medium', 'long'])
df.groupby('trip_distance_group')['passenger_count'].mean().sort_values(ascending=False)

<ipython-input-28-5c67c87d9537>:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('trip_distance_group')['passenger_count'].mean().sort_values(ascending=False)


,passenger_count
trip_distance_group,
long,1.590035
medium,1.585319
short,1.558400


#### 使用 Pandas 的 pd.cut() 函数将 trip_distance（行程距离）列分成不同的组别，并创建一个新的分类列 trip_distance_group